**AGENT-BASED ARTIFICIAL INTELLIGENCE, POLIBA, A.Y. 2023/2024**

**Solving Problems by Searching with INFORMED STRATEGIES**

List of Contributors:
- Tommaso Di Noia
- Alberto Carlo Maria Mancino
- Vincenzo Paparella

## What we have done so far


So far, we have designed an agent able to reach a city starting from another city within a map through search algorithms, i.e., tree and graph search.

We have:
- modeled the environment;
- implemented a Problem Class that practically models the agent;
- implemented a Node Class that is exploited in the tree/graph search;
- implemented a Tree/Graph search class to perform the search.


Beyond the positions of the cities within map, no other information were available. Indeed, the search was guided by uninformed strategies solely.

## The environment with additional information

To introduce the informed strategies for search, we now consider some additional information within the map.

We suppose to know the coordinates of the cities that we store in a dictionary.

In [1]:
streets = {
    'Andria': ['Corato', 'Trani'],
    'Corato': ['Ruvo', 'Trani', 'Andria', 'Altamura'],
    'Altamura': ['Corato', 'Ruvo', 'Modugno'],
    'Ruvo': ['Corato', 'Bisceglie', 'Terlizzi', 'Altamura'],
    'Terlizzi': ['Ruvo', 'Molfetta', 'Bitonto'],
    'Bisceglie': ['Trani', 'Ruvo', 'Molfetta'],
    'Trani': ['Andria', 'Corato', 'Bisceglie'],
    'Molfetta': ['Bisceglie', 'Giovinazzo', 'Terlizzi'],
    'Giovinazzo': ['Molfetta', 'Modugno', 'Bari', 'Bitonto'],
    'Bitonto': [ 'Modugno', 'Giovinazzo', 'Terlizzi'],
    'Modugno': ['Bitonto', 'Giovinazzo', 'Bari', 'Altamura'],
    'Bari': ['Modugno', 'Giovinazzo']
}

# cities coordinates
streets_coords = {
    'Andria': (41.2316, 16.2917),
    'Corato': (41.1465, 16.4147),
    'Altamura': (40.8302, 16.5545),
    'Ruvo': (41.1146, 16.4886),
    'Terlizzi': (41.1321, 16.5461),
    'Bisceglie': (41.243, 16.5052),
    'Trani': (41.2737, 16.4162),
    'Molfetta': (41.2012, 16.5983),
    'Giovinazzo': (41.1874, 16.6682),
    'Bitonto': (41.1118, 16.6902),
    'Modugno': (41.0984, 16.7788),
    'Bari': (41.1187, 16.852)
}

Once we know these coordinates, we are able to compute the distance between two cities through a function.

For this reason, we now decide to model the environment within a Roads Class. In this class, we have the cities and their coordinates as attributes. In addition, we have a distance method that returns the distance in kilometers between two cities.



In [2]:
class Roads:
    def __init__(self, streets, coordinates):
        self.streets = streets
        self.coordinates = coordinates

    def distance(self, start, end):
        lat_a, long_a = self.coordinates[start]
        lat_b, long_b = self.coordinates[end]
        lat_diff = abs(lat_a - lat_b)*111  # <- *111 to just convert the latitude distance in KM.
        long_diff = abs(long_a - long_b)*111 # <- *111 to just convert the longitude distance in KM.
        return math.sqrt(lat_diff**2 + long_diff**2)

## Slightly changing our agent

We have changed our environment to support the additional information. Consequently, we must slightly adapt our agent to this new environment. Indeed, we know have a class, instead of a simple dictionary. Then, we modify our StreetProblem class.

Specifically, given a state, we gather the possible actions that the agent can perform from the street attribute of the environment. Then, we must change the actions function!

Finally, we can now consider the cost of performing an action starting from a state! The cost is the distance in kilometers of performing the action of reaching a city starting from another city. Therefore, we exploit the distance function implemented in the environment.

In [3]:
class StreetProblem:

    def __init__(self, initial_state, goal_state, environment):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.environment = environment

    def successors(self, state):
        """
        Given a state returns the reachable states with the respective actions
        :param state: actual state
        :return: list of successor states and actions
        """
        possible_actions = self.actions(state)
        return [(self.result(state, a), a) for a in possible_actions]

    def actions(self, state):
        """
        Given a state returns the list of possible actions
        :param state: actual state
        :return: a list of actions
        """
        return self.environment.streets[state] # <- WE HAVE MODIFIED HERE

    def result(self, state=None, action=None):
        """
        Given a state and an action returns the reached state
        :param state: actual state
        :param action: chosen action
        :return: reached state
        """
        return action

    def goal_test(self, state):
        """
        Checks if the goal condition has been reached
        :param state: actual state
        :return: True if the goal condition is matched, False otherwise
        """
        return state == self.goal_state

    def cost(self, state, action):  # <- WE HAVE MODIFIED THIS FUNCTION
        """
        Given a state and an action returns the cost of the action
        :param state: a state
        :param action: an action
        :return: the cost of doing that action in that state
        """
        reached_state = self.result(state, action)
        return self.environment.distance(state, reached_state)

## Slightly changing our Tree/Graph Search

We can now compute the cost of performing an action from a state thanks to the cost function in the Problem class.

Consequently, we modify the TreeSearch and GraphSearch classes to consider such costs when expanding a node.

In [4]:
class TreeSearch:
    """
    A class able to find a solution with a given search strategy
    """

    def __init__(self, problem, strategy=None):
        self.problem = problem
        self.strategy = strategy
        self.fringe = []

    def __repr__(self):
        return 'Tree Search'

    def run(self):
        """
        Run the search
        :return: a path or a failure
        """

        node = Node(state=self.problem.initial_state,
                    parent=None,
                    action=None,
                    cost=0,
                    depth=0)

        # search loop
        while True:
            # check if the node passes the goal test
            if self.problem.goal_test(node.state):
                return 'Ok', node

            # expand the node
            new_states = self.problem.successors(node.state)
            new_nodes = [node.expand(state=s,
                                     action=a,
                                     cost=self.problem.cost(node.state, a)) for s, a in new_states]  # WE HAVE ADDED THE COST HERE (INSTEAD OF SIMPLY 1)

            # update the fringe
            self.fringe = self.fringe + new_nodes

            # check if the search fails: empty fringe, unless the last node from the fringe contains the goal state
            # if the fringe is not empty, we pop the next node from the fringe according to the strategy
            if len(self.fringe) != 0:
                self.fringe, node = self.strategy.select(self.fringe)
                # check to manage if the fringe becomes empty within the strategy class (e.g., because of DepthLimited)
                if node is None:
                    return 'Fail', []
            else:
                if self.problem.goal_test(node.state):
                    return 'Ok', node
                else:
                    return 'Fail', []


In [5]:
class GraphSearch:
    """
    A class able to find a solution with a given search strategy
    """

    def __init__(self, problem, strategy=None):
        self.problem = problem
        self.strategy = strategy
        self.fringe = []
        self.visited = []  # <- The list containing the visited states

    def __repr__(self):
        return 'Graph Search'

    def run(self):
        """
        Run the search
        :return: a path or a failure
        """

        node = Node(state=self.problem.initial_state,
                    parent=None,
                    action=None,
                    cost=0,
                    depth=0)

        # search loop
        while True:
            # check if the node passes the goal test
            if self.problem.goal_test(node.state):
                return 'Ok', node

            # add visited for the graph search
            self.visited.append(node.state)

            # expand the node
            new_states = self.problem.successors(node.state)
            new_nodes = [node.expand(state=s,
                                     action=a,
                                     cost=self.problem.cost(node.state, a)) for s, a in new_states]  # WE HAVE ADDED THE COST HERE (INSTEAD OF SIMPLY 1)

            # we retain the states not already visited!
            new_nodes = [n for n in new_nodes if n.state not in self.visited]
            self.fringe = [n for n in self.fringe if n.state not in self.visited]

            # update the fringe
            self.fringe = self.fringe + new_nodes

            # check if the search fails: empty fringe, unless the last node from the fringe contains the goal state
            # if the fringe is not empty, we pop the next node from the fringe according to the strategy
            if len(self.fringe) != 0:
                self.fringe, node = self.strategy.select(self.fringe)
                # check to manage if the fringe becomes empty within the strategy class (e.g., because of DepthLimited)
                if node is None:
                    return 'Fail', []
            else:
                if self.problem.goal_test(node.state):
                    return 'Ok', node
                else:
                    return 'Fail', []


## Evaluation and Heuristic functions

From the theory, informed search strategies exploit **evaluation functions** _f(n)_ as additional information to more efficiently estimate the sequence of actions to reach a goal state. The evaluation functions depend on the informed strategy we adopt.

Specifically, most of the times, an evaluation function includes an a **heuristic function** _h(n)_ that represents the estimated cost of the cheapest path from the state at node _n_ to a goal state.

Which heuristic function could we use in our map problem? In this problem, the _straight line distance_ is a good candidate to be our heuristic. Indeed, the straight line distance is the shortest possible distance to reach a goal state starting from a given state.

Where do we implement such heuristic function? Intuitively, the heuristic function depends on the problem we are dealing with. Therefore, we add the method _h(state)_ in the StreetProblem class that returns the straight line distance from a state in input to the goal state.

In [6]:
import math


class StreetProblem:

    def __init__(self, initial_state, goal_state, environment):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.environment = environment

    def successors(self, state):
        """
        Given a state returns the reachable states with the respective actions
        :param state: actual state
        :return: list of successor states and actions
        """
        possible_actions = self.actions(state)
        return [(self.result(state, a), a) for a in possible_actions]

    def actions(self, state):
        """
        Given a state returns the list of possible actions
        :param state: actual state
        :return: a list of actions
        """
        return self.environment.streets[state]

    def result(self, state=None, action=None):
        """
        Given a state and an action returns the reached state
        :param state: actual state
        :param action: chosen action
        :return: reached state
        """
        return action

    def goal_test(self, state):
        """
        Checks if the goal condition has been reached
        :param state: actual state
        :return: True if the goal condition is matched, False otherwise
        """
        return state == self.goal_state

    def cost(self, state, action):
        """
        Given a state and an action returns the cost of the action
        :param state: a state
        :param action: an action
        :return: the cost of doing that action in that state
        """
        reached_state = self.result(state, action)
        return self.environment.distance(state, reached_state)

    def h(self, state):  # <- HEURISTIC FUNCTION
        lat_a, long_a = self.environment.coordinates[state]
        lat_b, long_b = self.environment.coordinates[self.goal_state]
        lat_diff = abs(lat_a - lat_b) * 111  # <- *111 to just convert the latitude distance in KM.
        long_diff = abs(long_a - long_b) * 111  # <- *111 to just convert the longitude distance in KM.
        return math.sqrt(lat_diff ** 2 + long_diff ** 2)

## Informed strategies

### Greedy Search

The Greedy search algorithm expands the node that appears to be closest to the goal. In other words, the evaluation function corresponds to the heuristic function, i.e., _f(n)=h(n)_.

Then, we sort the nodes according to their heuristic function score. Finally, we take the first node of the frontier (the one having the minimum heuristic function score, i.e., the closest to the goal state).

In [7]:
class Greedy:
    def __init__(self, problem):
        self.problem = problem

    def __repr__(self):
        return 'Greedy strategy'

    def select(self, fringe):
        # sort fringe following the evaluation function
        fringe = sorted(fringe, key=lambda x: self.problem.h(x.state))
        node = fringe.pop(0)
        return fringe, node

### AStar Search

The AStar search algorithm expands a node avoiding expanding paths that are already expensive. The evaluation function combines with the sum the heuristic function and the cost so far to reach the node _n_, i.e., _f(n)=g(n)+h(n)_.

Then, we sort the nodes according to their evaluation function score. Finally, we take the first node of the frontier (the one having the minimum evaluation function score, i.e., the closest to the goal state).

To ensure optimality in AStar search strategy, we must consider some conditions regarding the heuristic:
1. in Tree Search, the heuristic must be admissible. A heuristic function is admissible if it never over-estimates the cost to reach the goal state. Moreover, it must be _h(n)>=0_, so that _h(G)=0_ for any goal _G_.
2. in Graph Search, the heuristic must be consistent. A heuristic function is consistent if it respects the triangle inequality property. This means that the single number _h(n)_ should be less than the cost _c(n, a, n')_ of performing the action from _n_ to _n'_ plus the heuristic score _h(n')_.

In our case, the previous implemented heuristic function (the straightline distance) is consistent (and then admissible). Then, we are safe!

Let's implement the AStar search strategy.

In [8]:
class AStar:
    def __init__(self, problem):
        self.problem = problem

    def __repr__(self):
        return 'AStar strategy'

    def select(self, fringe):
        # sort fringe following the evaluation function
        fringe = sorted(fringe, key=lambda x: (self.problem.h(x.state)+x.cost))
        node = fringe.pop(0)
        return fringe, node

## Execution

Let's compute our main.

In [9]:
class Node:
    def __init__(self, state, parent, action, cost, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.depth = depth

    def __repr__(self):
        """
        A representation of the class. Useful with functions like print.
        :return: a string
        """
        return f'({self.state})'

    def expand(self, state, action, cost=1):
        """
        Given a new state returns a child tree node containing that state
        :param new_state: state that will be contained by the node
        :param action: action that led to the state
        :param cost: cost of the path of the previous node
        :return: a child node
        """
        return Node(state=state,
                    parent=self,
                    action=action,
                    cost=self.cost+cost,
                    depth=self.depth+1)

    def path(self):
        """
         Returns the path from the root node to the actual node
        :return: a list of actions
        """
        path = []
        node = self
        while node.parent:
            path.append(node.action)
            node = node.parent
        path = list(reversed(path))
        return path


In [10]:
map = Roads(streets, streets_coords)
# formulate the problem
initial_state = 'Ruvo'
goal_state = 'Bari'
map_problem = StreetProblem(environment=map,
                      initial_state=initial_state,
                      goal_state=goal_state)

# search strategy
strategies = [AStar(map_problem), Greedy(map_problem)]

# search algorithm (Tree Search / Graph Search)
for strategy in strategies:
    search = TreeSearch(problem=map_problem, strategy=strategy)
    result, node = search.run()
    print(f'{strategy}, {search}')
    print(result)
    print(node.path())
    print(node.cost)

print("---------")

strategies = [AStar(map_problem), Greedy(map_problem)]
for strategy in strategies:
    search = GraphSearch(problem=map_problem, strategy=strategy)
    result, node = search.run()
    print(f'{strategy}, {search}')
    print(result)
    print(node.path())
    print(node.cost)

AStar strategy, Tree Search
Ok
['Terlizzi', 'Bitonto', 'Modugno', 'Bari']
41.20288964555017
Greedy strategy, Tree Search
Ok
['Terlizzi', 'Bitonto', 'Modugno', 'Bari']
41.20288964555017
---------
AStar strategy, Graph Search
Ok
['Terlizzi', 'Bitonto', 'Modugno', 'Bari']
41.20288964555017
Greedy strategy, Graph Search
Ok
['Terlizzi', 'Bitonto', 'Modugno', 'Bari']
41.20288964555017
